In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objs as go
# import dash_core_components as dcc
import sklearn
import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the kg Data
df_kg= pd.read_csv("/kaggle/input/covid19-healthy-diet-dataset/Fat_Supply_Quantity_Data.csv")
df_kg.head()

In [ ]:
# Load the Kcal Data
df_kcal= pd.read_csv("/kaggle/input/covid19-healthy-diet-dataset/Food_Supply_kcal_Data.csv")
df_kcal.head()

# EDA - KCal Data Set

In [ ]:
# Drop all null/NAN values
df_kcal= df_kcal.dropna()
# drop duplicates
df_kcal= df_kcal.drop_duplicates()
# Drop Unit Column
df_kcal = df_kcal.drop('Unit (all except Population)', axis=1)
# Increase the Range - by multiplying by 2
df_kcal.iloc[:,1:24]= df_kcal.iloc[:,1:24] *2
# For column - Undernourished - As we have Value content - "<2.5" so will be replaced by 2.5
df_kcal['Undernourished'] = df_kcal.apply(lambda row: 2.5 if row['Undernourished'] == '<2.5' else float(row['Undernourished']), axis = 1)

In [ ]:
df_kcal.info()

In [ ]:
df_kcal.describe().T

In [ ]:
fig = px.scatter(df_kcal[df_kcal.Country != 'Yemen'], x="Obesity", y = "Confirmed", size = "Active", hover_name='Country', log_x=False,
                 size_max=30, template="simple_white")

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=0,
            y0=df_kcal[df_kcal.Country != 'Yemen']['Confirmed'].mean(),
            x1=df_kcal[df_kcal.Country != 'Yemen']['Obesity'].max(),
            y1=df_kcal[df_kcal.Country != 'Yemen']['Confirmed'].mean(),
            line=dict(
                color="crimson",
                width=4
            ),
    )

fig.show()


In [ ]:
plt.scatter(df_kcal.Obesity, df_kcal.Recovered, c ="pink")
plt.hlines(y= df_kcal.Recovered.mean(),xmin = df_kcal[df_kcal.Country != 'Yemen']['Obesity'].min(), xmax = df_kcal[df_kcal.Country != 'Yemen']['Obesity'].max())
plt.xlabel("Obesity") 
plt.ylabel("Recovered") 
plt.show() 

In [ ]:
test_df_high= df_kcal.sort_values("Recovered",ascending=False).head(10)
test_df_low= df_kcal.sort_values("Recovered",ascending=False).tail(10)
df_h=test_df_high["Recovered"]
df_l=test_df_low["Recovered"]

import scipy.stats as stats
output,pval=stats.ttest_ind(a=test_df_high['Animal Products'], b=test_df_low['Animal Products'])
output,pval

In [ ]:
import seaborn as sns 
sns.distributions.displot(df_h,df_l,kind="kde")
sns.distributions.displot(df_l,kind = "kde")

In [ ]:
stats.ttest_ind(a=test_df_low['Animal Products'], b=test_df_high['Animal Products'], equal_var=True)

In [ ]:
stats.ttest_ind(a=test_df_low['Alcoholic Beverages'], b=test_df_high['Alcoholic Beverages'], equal_var=True)

In [ ]:
stats.ttest_ind(a=test_df_low['Milk - Excluding Butter'], b=test_df_high['Milk - Excluding Butter'], equal_var=True)

In [ ]:
stats.ttest_ind(a=test_df_low['Obesity'], b=test_df_high['Obesity'], equal_var=True)

In [ ]:
stats.ttest_ind(a=test_df_low['Starchy Roots'], b=test_df_high['Starchy Roots'], equal_var=True)

In [ ]:
stats.ttest_ind(a=test_df_low['Cereals - Excluding Beer'], b=test_df_high['Cereals - Excluding Beer'], equal_var=True)

In [ ]:
stats.ttest_ind(a=test_df_low['Vegetal Products'], b=test_df_high['Vegetal Products'], equal_var=True) 

In [ ]:
# Highest Recovered Rate - Belgium, United States of America, Serbia, Sweden is 0, French Polynesia,Kiribati,Korea, North, Myanmar,New Caledonia and Turkmenistan
df_kcal.sort_values(by=['Recovered'],ascending=False).tail(15)[['Country','Recovered','Confirmed','Deaths']]

# Data Visualization and Analysis - KCal Dataset

In [ ]:
# Distributions
fig = px.bar(df_kcal, x = "Country", y ="Recovered").update_xaxes(categoryorder="total descending")
fig.show()

# Recovered Rate :
1. Belgium, United States of America, Serbia and Sweden is 0
2. French Polynesia,Kiribati,Korea, North, Myanmar,New Caledonia and Turkmenistan Null Content
3. Least Recovered : United Republic of Tanzania	0.000306 - Recovered Rate and 0.000852 - Confimed Rate.  Difference = 0.000546
4. Highest recovery rate - Montenegro, Czechia,Luxembourg, Slovenia are European Countries
5. Lowest recovery rate - with 0 Death Rate are Islands. -

In [ ]:
# Confirmed VS Deaths - keeping Active cases into Consideration of all the Country
fig = px.scatter(df_kcal, x="Recovered", y = "Deaths",size = "Active", hover_name='Country', log_x=False,
                 size_max=30, trendline = "ols", marginal_x = "box",marginal_y = "violin", template="simple_white")
fig.show()

In [ ]:
# European countries with high recover rate
df_hr = df_kcal.sort_values(by=['Recovered'],ascending=False).head(4)
fig = px.pie(values = df_hr[animal_features].mean().tolist(), names = animal_features,
             title='Mean food intake by Animal products groups - High Recovered European Countries')
fig.show()

fig = px.pie(values = df_hr[vegetal_features].mean().tolist(), names = vegetal_features,
             title='Mean food intake by Vegtal products groups - High Recovered European Countries')
fig.show()

In [ ]:
df_lw

In [ ]:
# European countries with high recover rate
feature = ['Alcoholic Beverages', 'Cereals - Excluding Beer', 'Starchy Roots','Animal fats', 'Meat','Milk - Excluding Butter']
df_lw = df_kcal.sort_values(by=['Recovered'],ascending=False).tail(10)
fig = px.pie(values = df_lw[feature].mean().tolist(), names = feature,title='Mean food intake  - Low Recovered Countries')
fig.show()
df_hw = df_kcal.sort_values(by=['Recovered'],ascending=False).head(5)
fig = px.pie(values = df_hw[feature].mean().tolist(), names = feature,
             title='Mean food intake - High Recovered Countries')
fig.show()

# For European Country with Highest Recovery Rate :
1. Under Animal Product : Intake of Meat and Milk is more 40% and 35% 
2. Under Vegetal Product : Intake of Cereals is way more than any other products - 42%

In [ ]:
# Animal Products: Which products make the difference between High and Low Recovered countries ?
df_high_recover = df_kcal[df_kcal.Recovered > df_kcal['Recovered'].mean()]
df_low_recover = df_kcal[df_kcal.Recovered <= df_kcal['Recovered'].mean()]

In [ ]:
# Classify - Animal Features and Vegetal Features
animal_features = ['Animal fats', 'Aquatic Products, Other', 'Eggs', 'Fish, Seafood', 'Meat',
                   'Milk - Excluding Butter', 'Offals']
vegetal_features = ['Alcoholic Beverages', 'Cereals - Excluding Beer', 'Fruits - Excluding Wine', 'Miscellaneous', 'Oilcrops', 'Pulses',
                    'Spices', 'Starchy Roots', 'Stimulants', 'Sugar & Sweeteners', 'Sugar Crops', 'Treenuts',
                    'Vegetable Oils', 'Vegetables']

In [ ]:
# # Classify - Animal Features and Vegetal Features
# animal_features = ['Animal fats', 'Aquatic Products, Other','Alcoholic Beverages',  'Meat', 'Offals']
# # 'Milk - Excluding Butter','Fish, Seafood','Eggs',
# vegetal_features = ['Cereals - Excluding Beer', 'Fruits - Excluding Wine', 'Miscellaneous', 'Oilcrops', 'Pulses',
#                     'Spices', 'Starchy Roots', 'Stimulants', 'Sugar & Sweeteners', 'Sugar Crops', 'Treenuts',
#                     'Vegetable Oils', 'Vegetables']

In [ ]:
# High Recovered rates countries : Animal products
fig = px.pie(values = df_high_recover[animal_features].mean().tolist(), names = animal_features,
             title='Mean food intake by Animal products groups - High Recovered Countries')
fig.show()

In [ ]:
fig = px.pie(values = df_low_recover[animal_features].mean().tolist(), names = animal_features,
             title='Mean food intake by Animal products groups - Low Recovered Countries')
fig.show()

Analysis: The distributions are somewhat similar. The order of highest to lowest intake is the same (except for Meat and Animal fats). However, two things stand out: The Meat contribution is huge; almost 38% - first group and second group 44%. The difference between the Fish, Seafood intake in both groups (the first - around 5% , the second - around 9% ) Same with Milk - (first-35% and second-31% ).On exceeding the Limit of Milk intake recovery rate is going down. 

In [ ]:
# Vegetal Products
fig = px.pie(values = df_high_recover[vegetal_features].mean().tolist(), names = vegetal_features,
             title='Mean food intake by Vegetal products groups - High Recovered Countries')
fig.show()

fig = px.pie(values = df_low_recover[vegetal_features].mean().tolist(), names = vegetal_features,
             title='Mean food intake by Vegetal products groups - Low Recovered Countries')
fig.show()

The intake of Starchy Roots in Low Recovered Countries is almost 10% , double that of High Recovered Countries. The intake of Alcoholic Beverages is double in High Recovered Countries(5%), as in Low Recovered Countries it reaches almost 2.4% .

In [ ]:
# demo.Recovered.mean()
df_kcal['RecoveredAboveAverage'] = (df_kcal["Recovered"] > df_kcal['Recovered'].mean()).astype(int)

In [ ]:
fig = px.scatter(df_kcal, x = 'Animal Products', y ='Vegetal Products',
                 color='RecoveredAboveAverage', hover_name = 'Country')
fig.show()

Analysis: we see there is a relation between high consuption of Animal Products (comparing with Vegetal Products) and high obesity rates. Using the hover information you can find the country with highest Animal Products intake (Iceland) and the one with highest Vegetal Products intake (Nigeria)

In [ ]:
fig = px.bar(df_kcal, x = "Country", y ="Deaths", facet_col = "RecoveredAboveAverage")
fig.update_xaxes(matches=None,categoryorder="total descending")
fig.show()

In the figure above, we can see clearly that the "high Recovered rate" countries have a least impact from COVID-19.

In [ ]:
fig = px.scatter_matrix(df_kcal[['Meat', 'Milk - Excluding Butter', 'Fish, Seafood',
                         'Cereals - Excluding Beer','Deaths','Recovered']])
fig.show()

So, we are interested in the last "row" of this matrix. Nothing seems particularly linear, but we'll see what we can tell from building linear models.

Diet vs Covid

In [ ]:
corr_food=df_kcal.loc[:, df_kcal.columns != 'RecoveredAboveAvg'].corr(method='pearson')
corr_final=corr_food.abs().unstack().sort_values(ascending = False)
corr_final.drop(corr_final.head(32).index, inplace=True)
corr_confirmed = corr_final['Confirmed'].head(15)
corr_confirmed = corr_confirmed.drop(['Recovered', 'Deaths', 'Active', 'Undernourished', 'Obesity'])
corr_deaths = corr_final['Deaths'].head(15)
corr_deaths = corr_deaths.drop(['Recovered', 'Confirmed', 'Active', 'Undernourished', 'Obesity'])
corr_recovered = corr_final['Obesity'].head(14)
corr_recovered = corr_recovered.drop(['Confirmed', 'Deaths', 'Undernourished', 'Recovered'])

In [ ]:
corr_heatmap=df_kcal[['Deaths','Animal Products','Animal fats','Cereals - Excluding Beer','Eggs','Meat','Milk - Excluding Butter','Pulses','Starchy Roots','Sugar & Sweeteners','Vegetal Products']]
x=corr_heatmap.corr(method='pearson')
fig = go.Figure(ff.create_annotated_heatmap(z=x[['Deaths']].sort_values(by=['Deaths'],ascending=False).values, x = ['Deaths'], y=x[['Deaths']].sort_values(by=['Deaths'],ascending=False).index.to_list(), colorscale='Viridis'))
fig.show()

corr_heatmap=df_kcal[['Confirmed','Animal Products','Animal fats','Cereals - Excluding Beer','Eggs','Meat','Milk - Excluding Butter','Pulses','Starchy Roots','Sugar & Sweeteners','Vegetal Products']]
x=corr_heatmap.corr(method='pearson')
fig = go.Figure(ff.create_annotated_heatmap(z=x[['Confirmed']].sort_values(by=['Confirmed'],ascending=False).values, x = ['Confirmed'], y=x[['Confirmed']].sort_values(by=['Confirmed'],ascending=False).index.to_list(), colorscale='Viridis'))
fig.show()

corr_heatmap=df_kcal[['Recovered','Animal Products','Animal fats','Cereals - Excluding Beer','Eggs','Meat','Milk - Excluding Butter','Pulses','Starchy Roots','Sugar & Sweeteners','Vegetal Products']]
x=corr_heatmap.corr(method='pearson')
fig = go.Figure(ff.create_annotated_heatmap(z=x[['Recovered']].sort_values(by=['Recovered'],ascending=False).values, x = ['Recovered'], y=x[['Recovered']].sort_values(by=['Recovered'],ascending=False).index.to_list(), colorscale='Viridis'))
fig.show()

Obesity has a stronger correlation with covid deaths than recovery and undernourished patients has a stronger correlation with covid recovery than deaths. This could mean that in average, obese patients are most likely to die from covid while undernourished are most likely to survive.

In [ ]:
# Healthy Diet vs covid
corr_heatmap=df_kcal[['Deaths','Confirmed','Recovered','Obesity','Undernourished']]
x=corr_heatmap.corr(method='pearson').round(3)
ff.create_annotated_heatmap(z=x.values, x=x.columns.to_list(), y=x.columns.to_list(), colorscale='Viridis', showscale=True)

Indeed, we can now see that Recovered has a stronger correlation with Obesity than Undernourished and Deaths patients has a stronger correlation with Obesity than Undernourished. This could mean that in average, obese patients are most likely to recover from covid while undernourished are most likely to die. This is why obesity worsens outcomes from covid.

In [ ]:
# Recover average diet
recover_set = df_kcal[df_kcal['Recovered'] == df_kcal['Recovered']].sort_values(by='Recovered', ascending=False).head(10)
# save the desc
recover_mean = recover_set.describe().iloc[1]
recover_mean = pd.DataFrame(recover_mean).drop(['Deaths', 'Population','Undernourished','Obesity', 'Recovered', 'Confirmed', 'Active'], axis=0)
recover_mean = recover_mean.sort_values(by='mean', ascending=False).iloc[:11]
recover_mean

Vegetal Products and Cereals contribution is more in Recovery Rate. 

In [ ]:
# Confirmed Cases - animal products , milk without butter, obesity , starchy roots, vegetal products 
# Death Rates

The pie chart above looks a lot like the average pie chart diet we made earlier for the world consumption. The countries with the most recovered rate seem to consume more vegetables than people on average.

In [ ]:
# Undernutrition average diet
undernutrition_set = demo[demo['Undernourished'] == demo['Undernourished']].sort_values(by='Undernourished', ascending=False).head(10)
undernutrition_mean = undernutrition_set.describe().iloc[1]
undernutrition_mean = pd.DataFrame(undernutrition_mean).drop(['Deaths', 'Population','Undernourished','Obesity', 'Recovered', 'Confirmed', 'Active',], axis=0)
undernutrition_mean = undernutrition_mean.sort_values(by='mean', ascending=False).iloc[:11]

In [ ]:
fig = px.pie(values = undernutrition_mean['mean'].values, names = undernutrition_mean.index.tolist(),)
fig.show()

Here undernourished people consume way less animal products and much more starchy roots than the world's consumption in average or the obese people on average. Moreover, they seem to be consuming a bit more alcoholic beverages.

In [ ]:
# Obesity average Diet
obesity_set = demo[demo['Obesity'] == demo['Obesity']].sort_values(by='Obesity', ascending=False).head(10)
obesity_mean = obesity_set.describe().iloc[1]
obesity_mean = pd.DataFrame(obesity_mean).drop(['Deaths', 'Population','Undernourished','Obesity', 'Recovered', 'Confirmed', 'Active'], axis=0)
obesity_mean = obesity_mean.sort_values(by='mean', ascending=False).iloc[:11]


fig = px.pie(values = obesity_mean['mean'].values, names = obesity_mean.index.tolist(),
             )
fig.show()

The pie chart above looks a lot like the average pie chart diet we made earlier for the world consumption. The countries with the most obesity rate seem to consume more vegetables than people on average.


In [ ]:
class_df =demo
class_df['Veg']= (class_df[['Alcoholic Beverages','Fruits - Excluding Wine','Oilcrops','Vegetal Products', 'Vegetable Oils','Vegetables','Milk - Excluding Butter','Pulses', 'Spices', 'Starchy Roots','Stimulants', 'Sugar Crops','Miscellaneous','Sugar & Sweeteners', 'Treenuts','Cereals - Excluding Beer']]).sum(axis=1)
class_df['Non-veg'] = (class_df[['Animal Products', 'Animal fats','Aquatic Products, Other','Eggs','Fish, Seafood','Meat','Offals']]).sum(axis=1)
class_df[['Veg', 'Non-veg']]

In [ ]:
class_df.drop(['Alcoholic Beverages', 'Fruits - Excluding Wine', 'Oilcrops',
'Vegetal Products', 'Vegetable Oils',
'Vegetables','Milk - Excluding Butter','Pulses', 'Spices', 'Starchy Roots',
'Stimulants', 'Sugar Crops','Miscellaneous',
'Sugar & Sweeteners', 'Treenuts','Cereals - Excluding Beer',

'Animal Products', 'Animal fats',
'Aquatic Products, Other','Eggs',
'Fish, Seafood',  'Meat','Offals'],axis=1, inplace=True)
class_df

In [ ]:
# Rearrange the columns
class_df = class_df[["Country","Veg","Non-veg","Obesity","Undernourished", "Confirmed","Deaths","Recovered","Active","Population"]]
class_df

In [ ]:
# Lowest Death Rate
class_df.sort_values(by=['Deaths'],ascending=True).head(10)[['Country','Veg','Non-veg','Deaths']]

In [ ]:
# Highest Death Rate
class_df.sort_values(by=['Deaths'],ascending=False).head(10)[['Country','Veg','Non-veg','Deaths']]

In [ ]:
# Highest Recovered Rate
class_df.sort_values(by=['Recovered'],ascending=False).head(10)[['Country','Veg','Non-veg','Recovered']]

In [ ]:
# Lowest Recovered Rate
class_df.sort_values(by=['Recovered'],ascending=True).head(10)[['Country','Veg','Recovered']]

In [ ]:
df_Deaths = class_df.sort_values(by="Deaths",ascending=False)
df_Deaths.head()

In [ ]:
# Considering Veg vs Deaths 
df_Deaths = class_df.sort_values(by="Deaths",ascending=True)
df_Deaths.head()

plt.figure(figsize=(50,5))
plt.subplot(1,2,1)

plt.plot(df_Deaths["Deaths"],df_Deaths["Veg"], label="Veg")
plt.plot(df_Deaths["Deaths"],df_Deaths["Non-veg"], label="Non-veg")

# plt.plot(df_Deaths["Recovered"],df_Deaths["Veg"], label= "Veg")
# plt.plot(df_Deaths["Recovered"],df_Deaths["Non-veg"], label= "Non-veg")

# plt.plot(df_Deaths["Deaths"],df_Deaths["Obesity"], label= "Recovered")

plt.title('Deaths Vs veg and non-veg Consumption')
plt.xlabel('Consumption')
plt.ylabel('Deaths cases')
plt.legend(loc='best')
plt.show()

In [ ]:
# Considering Veg vs Deaths 
df_Recovered = class_df.sort_values(by="Recovered",ascending=False)
df_Recovered.head()

plt.figure(figsize=(50,5))
plt.subplot(1,2,1)

plt.plot(df_Recovered["Recovered"],df_Recovered["Veg"], label="Veg")
plt.plot(df_Recovered["Recovered"],df_Recovered["Non-veg"], label="Non-veg")

plt.title('Recovered Vs veg and non-veg Consumption')
plt.xlabel('Consumption')
plt.ylabel('Recovered cases')
plt.legend(loc='best')
plt.show()

The above analysis states that the country with more veg content is Recovered Rate is more and Death Rate is Less.
- Meat , offale - Red meat contain are more prone to 

In [ ]:
demo.sum(axis=1)

In [ ]:
# pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(demo, title='kcal')
profile


In [ ]:
sorted_pairs= demo.corr().unstack().sort_values(kind="quicksort")
sorted_pairs

In [ ]:
negative_pairs = sorted_pairs[sorted_pairs < 0]
negative_pairs

In [ ]:
strong_pairs = sorted_pairs[(sorted_pairs) > 0.5]
strong_pairs

In [ ]:
demo.sort_values(by=['Deaths'],ascending=False).head(10)[['Country','Population','Recovered','Deaths']]

In [ ]:
demo.corrwith(demo["Deaths"]).sort_values()

In [ ]:
import seaborn as sns
sns.pairplot(df_kcal)

In [ ]:
Feature vs Country - top 10 and bottom 10 Death rate

In [ ]:
demo.describe().T

In [ ]:
# Obesity vs Death
import plotly.express as px
fig = px.scatter(demo, x="Deaths", y = "Obesity", size = "Active", hover_name='Country', log_x=False,
                 size_max=30, template="simple_white")

fig.add_shape(
        # Line Horizontal
            type="line",
            x0=0,
            y0=demo['Obesity'].mean(),
            x1=demo['Deaths'].max(),
            y1=demo['Obesity'].mean(),
            line=dict(
                color="crimson",
                width=4
            ),
    )

fig.show()


In [ ]:
demo['Vegetal Products'].value_counts

In [ ]:
demo['Recovered'].median()